In [2]:
#Long-Short Investment optimal
#tuning with numerical method with optuna for grid search the best hyper parameters and iteration for backtest


from __future__ import annotations
import os
from typing import Dict, List, Optional, Tuple
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from sklearn.preprocessing import MinMaxScaler
from tvDatafeed import TvDatafeed, Interval
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import optuna
from optuna.samplers import TPESampler
from optuna.pruners import MedianPruner
from dataclasses import dataclass
import cvxpy as cp


def rsi(series: pd.Series, period: int = 14) -> pd.Series:
    delta = series.diff()
    gain = delta.clip(lower=0).ewm(alpha=1 / period, adjust=False).mean()
    loss = (-delta.clip(upper=0)).ewm(alpha=1 / period, adjust=False).mean()
    rs = gain / (loss + 1e-12)
    out = 100 - (100 / (1 + rs))
    return out.fillna(50.0)


def max_drawdown_from_equity(equity: pd.Series) -> float:
    roll_max = equity.cummax()
    dd = equity / roll_max - 1.0
    return float(dd.min())  # negative value

class LSTMModel(nn.Module):
    def __init__(self, input_size=2, hidden_layer_size=50, output_size=1, layers=1, dropout=0.0):
        super().__init__()
        self.lstm = nn.LSTM(
            input_size,
            hidden_layer_size,
            num_layers=layers,
            batch_first=True,
            dropout=dropout if layers > 1 else 0.0,
        )
        self.fc = nn.Linear(hidden_layer_size, output_size)

    def forward(self, x):
        out, _ = self.lstm(x)  # (B, T, H)
        y = self.fc(out[:, -1, :])  # (B, 1)
        return y

class LSTMRSITradingModel:
    def __init__(self,tickers: List[str],start_date: str,end_date: str,*,initial_balance: float = 1_000_000.0,sequence_length: int = 38,hidden_layer_size: int = 492,learning_rate: float = 0.0021,epochs: int = 10,lstm_layers: int = 1,
        dropout: float = 0.0,tx_cost_bps: float = 10.0,long_threshold: float = 0.0,short_threshold: float = 0.0,train_frac: float = 0.7,seed: int = 42,tv_username: Optional[str] = None,tv_password: Optional[str] = None,
        interval: Interval = Interval.in_4_hour,n_bars: int = 300,):

        self.tickers = tickers
        self.start_date = start_date
        self.end_date = end_date
        self.sequence_length = sequence_length
        self.hidden_layer_size = hidden_layer_size
        self.learning_rate = learning_rate
        self.epochs = epochs
        self.lstm_layers = lstm_layers
        self.dropout = dropout
        self.tx_cost_bps = tx_cost_bps
        self.long_threshold = long_threshold
        self.short_threshold = short_threshold
        self.train_frac = train_frac
        self.initial_balance = float(initial_balance)
        self.tv_username = tv_username or os.getenv("TV_Username")    #Replace Username Tradingview(TV)
        self.tv_password = tv_password or os.getenv("TV_password")    #Replace Password Tradingview(TV)
        self.interval = interval
        self.n_bars = n_bars
        self.cash_balance = float(initial_balance)
        self.shares_held = 0.0
        self.portfolio_values: Dict[str, List[float]] = {t: [] for t in tickers}
        self.portfolio_value: List[float] = []
        self.best_params_by_asset: Dict[str, dict] = {}
        self.best_metrics_by_asset: Dict[str, dict] = {}

        self.asset_info = {
            "XAUUSD": {"stock_name": "Gold Spot", "broker": "OANDA"},
            "XAGUSD": {"stock_name": "Silver Spot", "broker": "OANDA"},
            "ZW1!": {"stock_name": "Wheat Futures", "broker": "CBOT"},
            "TLI": {"stock_name": "TLI", "broker": "SET"},
            "GULF": {"stock_name": "GULF", "broker": "SET"},
            "KCE": {"stock_name": "KCE", "broker": "SET"},
            "PTTEP": {"stock_name": "PTTEP", "broker": "SET"},
            "TIPH": {"stock_name": "TIPH", "broker": "SET"},
            "CPAXT": {"stock_name": "CPAXT", "broker": "SET"},
            "MTI": {"stock_name": "MTI", "broker": "SET"},
            "UKOIL": {"stock_name": "Brent Spot", "broker": "FXCM"},
            "AOT": {"stock_name": "AOT", "broker": "SET"},
            "BRN1!": {"stock_name": "Brent Futures", "broker": "ICEEUR"},
            "S501!": {"stock_name": "SET50 Futures", "broker": "TFEX"},
        }

        np.random.seed(seed)
        torch.manual_seed(seed)

        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        print(f"Using {'GPU' if torch.cuda.is_available() else 'CPU'}")

       
        self.model = LSTMModel(input_size=2,hidden_layer_size=self.hidden_layer_size,output_size=1,layers=self.lstm_layers,dropout=self.dropout,).to(self.device)
        self.criterion = nn.MSELoss()
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=self.learning_rate)
        self.scaler = MinMaxScaler()
        self.data: Dict[str, pd.DataFrame] = {}
        self.predictions: np.ndarray = np.array([])
        self.actual_prices: np.ndarray = np.array([])
        self.px_t_oos: np.ndarray = np.array([])
        self.signals: np.ndarray = np.array([])
        self.t_oos_index: Optional[pd.Index] = None
        self.sig_colors = {1: "#2E7D32", 0: "#9E9E9E", -1: "#C62828"}
        self.sig_markers = {1: "triangle-up", 0: "circle", -1: "triangle-down"}
        self.line_colors = {"actual": "#1565C0", "pred": "#FB8C00", "equity": "#6A1B9A"}

    def current_params(self) -> Dict[str, float]:
        return {
            "sequence_length": self.sequence_length,
            "hidden_layer_size": self.hidden_layer_size,
            "lstm_layers": self.lstm_layers,
            "dropout": self.dropout,
            "learning_rate": self.learning_rate,
            "epochs": self.epochs,
            "tx_cost_bps": self.tx_cost_bps,
            "long_threshold": self.long_threshold,
            "short_threshold": self.short_threshold,
            "train_frac": self.train_frac,
        }

    def fetch_data(self) -> Dict[str, pd.DataFrame]:
        print(f"Fetching data for {self.tickers} using tvDatafeed...")
        try:
            if self.tv_username and self.tv_password:
                tv = TvDatafeed(username=self.tv_username, password=self.tv_password)
            else:
                tv = TvDatafeed()
        except Exception as e:
            raise RuntimeError(f"Failed to init TvDatafeed: {e}")

        self.data = {}
        for ticker in self.tickers:
            info = self.asset_info.get(ticker, {"stock_name": ticker, "broker": "OANDA"})
            exchange = info["broker"]
            try:
                print(f"Fetching {ticker} ({info['stock_name']}) from {exchange} ...")
                df = tv.get_hist(
                    symbol=ticker,
                    exchange=exchange,
                    interval=self.interval,
                    n_bars=self.n_bars,
                )
            except Exception as e:
                print(f"Error {ticker}@{exchange}: {e}")
                df = None

            if df is not None and not df.empty:
                if not pd.api.types.is_datetime64_any_dtype(df.index):
                    df.index = pd.to_datetime(df.index)
                df = df.sort_index()
                df = df.loc[self.start_date:self.end_date]
                self.data[ticker] = df
                print(f"OK: {ticker}, rows={len(df)}")
            else:
                print(f"No data: {ticker}@{exchange}")
        return self.data

    def prepare_data(self, asset: str):
        df = self.data[asset].copy()
        px = df["close"].astype(float)
        feat_rsi = rsi(px, 14)
        feats = pd.concat([px.rename("px"), feat_rsi.rename("rsi")], axis=1).dropna()


        split_idx = int(len(feats) * self.train_frac)
        if split_idx <= self.sequence_length + 1:
            raise ValueError(f"Not enough data for training split on {asset}")

        self.scaler = MinMaxScaler()
        self.scaler.fit(feats.iloc[:split_idx, :])
        feats_sc = pd.DataFrame(self.scaler.transform(feats), index=feats.index, columns=feats.columns)

        X, y, y_times = [], [], []
        L = self.sequence_length  
        for i in range(L, len(feats_sc) - 1):
            X.append(feats_sc.iloc[i - L : i, :].values)
            y.append(feats_sc.iloc[i + 1, 0])  
            y_times.append(feats.index[i + 1])

        X = np.asarray(X, dtype=np.float32)
        y = np.asarray(y, dtype=np.float32)
        y_times = pd.Index(y_times)


        n_train = max(1, split_idx - L - 1)
        self.X_train = torch.tensor(X[:n_train]).to(self.device)
        self.y_train = torch.tensor(y[:n_train]).to(self.device)
        self.X_test = torch.tensor(X[n_train:]).to(self.device)
        self.y_test = torch.tensor(y[n_train:]).to(self.device)
        self.t_oos_index = y_times[n_train:]
        px_arr = feats["px"].values
        self.px_t_oos = px_arr[L:-1][n_train:]

    def train_model(self):
        print(
            f"[train] seq={self.sequence_length} | hidden={self.hidden_layer_size} | "
            f"layers={self.lstm_layers} | dropout={self.dropout:.2f} | "
            f"lr={self.learning_rate:.6f} | epochs={self.epochs} | "
            f"thr=[long {self.long_threshold:.4f}, short {self.short_threshold:.4f}]"
        )
        self.model.train()
        log_every = max(10, self.epochs // 12)
        for epoch in range(self.epochs):
            self.optimizer.zero_grad()
            yhat = self.model(self.X_train).squeeze(-1)
            loss = self.criterion(yhat, self.y_train)
            loss.backward()
            self.optimizer.step()
            if (epoch + 1) % log_every == 0 or epoch == self.epochs - 1:
                print(f"Epoch {epoch+1}/{self.epochs}  TrainLoss: {loss.item():.8f}")

    def make_predictions(self):
        self.model.eval()
        with torch.no_grad():
            pred_sc = self.model(self.X_test).cpu().numpy()  # (N,1)
            true_sc = self.y_test.cpu().numpy().reshape(-1, 1)  # (N,1)
            zeros = np.zeros_like(pred_sc)
            self.predictions = self.scaler.inverse_transform(np.hstack([pred_sc, zeros]))[:, 0]
            self.actual_prices = self.scaler.inverse_transform(np.hstack([true_sc, zeros]))[:, 0]

    def generate_signals(self):
        exp_ret = (self.predictions - self.px_t_oos) / np.maximum(self.px_t_oos, 1e-12)
        tc = self.tx_cost_bps / 1e4
        sig = np.zeros_like(exp_ret)
        sig[exp_ret > (self.long_threshold + tc)] = -1
        sig[exp_ret < -(self.short_threshold + tc)] = 1
        for i in range(1, len(sig)):
            if sig[i] == 0:
                sig[i] = sig[i - 1]
        self.signals = sig.astype(int)
        return self.signals
    
    def simulate_trading(self, tx_cost_bps: float = 10.0, allow_short=False):
        cash = float(self.initial_balance)
        qty = 0.0
        tc_bps = self.tx_cost_bps if tx_cost_bps is None else float(tx_cost_bps)
        tc = tc_bps / 1e4
        pv = []

        for i in range(len(self.signals)):
            px_fill = float(self.actual_prices[i])
            sig = int(self.signals[i])

            if not allow_short:
                if sig == 1 and qty == 0.0:
                    qty = cash / (px_fill * (1 + tc))
                    cash -= qty * px_fill * (1 + tc)
                elif sig <= 0 and qty > 0.0:
                    cash += qty * px_fill * (1 - tc)
                    qty = 0.0
            else:
                if sig == 1:
                    if qty < 0: 
                        cash -= (-qty) * px_fill * (1 + tc)
                        qty = 0.0
                    if qty == 0.0:
                        qty = cash / (px_fill * (1 + tc))
                        cash -= qty * px_fill * (1 + tc)
                elif sig == -1:
                    if qty > 0: 
                        cash += qty * px_fill * (1 - tc)
                        qty = 0.0
                    if qty == 0.0:
                        qty = -(cash / (px_fill * (1 + tc)))
                        cash += (-qty) * px_fill * (1 - tc)
                else:
                    if qty > 0:
                        cash += qty * px_fill * (1 - tc)
                        qty = 0.0
                    elif qty < 0:
                        cash -= qty * px_fill * (1 + tc) 
                        qty = 0.0

            pv.append(cash + qty * px_fill)

        self.portfolio_value = pv
        return pv

    def calculate_performance(self, allow_short: bool = True, exec_next_bar: bool = True) -> Dict[str, float]:
        prices = np.asarray(self.actual_prices, dtype=float)
        signals = np.asarray(self.signals, dtype=int)
        n = min(len(prices), len(signals))
        if n == 0:
            return {
                "Final Portfolio Value (Cash)": float(self.initial_balance),
                "Total Profit (Cash)": 0.0,
                "Total Profit (Percentage)": 0.0,
                "Win Rate": 0.0,
                "Average Profit (Percentage)": 0.0,
                "Average Loss (Percentage)": 0.0,
                "Max Drawdown (Percentage)": 0.0,
                "Sigma (Period % stdev)": 0.0,
                "Num Trades": 0,
            }

        tc = (getattr(self, "tx_cost_bps", 0.0) or 0.0) / 1e4

        def fill_idx(i: int) -> int:
            return min(i + 1, n - 1) if exec_next_bar else i

        cash = float(self.initial_balance)
        qty = 0.0
        pos = 0 
        entry_cash = None
        trades_pnl = []
        pv = []

        for i in range(n):
            want = int(np.sign(signals[i]))  # normalize data to {-1,0,1}
            if not allow_short and want == -1:
                want = 0

            if pos == 0 and want != 0:
                k = fill_idx(i)
                px = prices[k]
                entry_cash = cash
                if want == 1:
                    qty = cash / (px * (1 + tc))
                    cash -= qty * px * (1 + tc)
                    pos = 1
                else:
                    qty = -cash / (px * (1 + tc))
                    cash += (-qty) * px * (1 - tc)
                    pos = -1

            elif pos == 1 and want != 1:
                k = fill_idx(i)
                px = prices[k]
                cash += qty * px * (1 - tc)
                pnl = cash - (entry_cash if entry_cash is not None else cash)
                trades_pnl.append(pnl)
                qty = 0.0
                pos = 0
                entry_cash = None
                if want == -1:
                    entry_cash = cash
                    qty = -cash / (px * (1 + tc))
                    cash += (-qty) * px * (1 - tc)
                    pos = -1

            elif pos == -1 and want != -1:
                k = fill_idx(i)
                px = prices[k]
                cash += qty * px * (1 + tc)
                pnl = cash - (entry_cash if entry_cash is not None else cash)
                trades_pnl.append(pnl)
                qty = 0.0
                pos = 0
                entry_cash = None
                if want == 1:
                    entry_cash = cash
                    qty = cash / (px * (1 + tc))
                    cash -= qty * px * (1 + tc)
                    pos = 1

            pv.append(cash + qty * prices[i])

        if pos != 0:
            px = prices[-1]
            if pos == 1:
                cash += qty * px * (1 - tc)
            else:
                cash += qty * px * (1 + tc)
            pnl = cash - (entry_cash if entry_cash is not None else cash)
            trades_pnl.append(pnl)
            qty = 0.0
            pos = 0
            pv[-1] = cash

        self.portfolio_value = pv
        pv_s = pd.Series(pv, dtype=float)
        ret = pv_s.pct_change().dropna()
        if ret.empty:
            final_value = float(pv_s.iloc[-1])
            return {
                "Final Portfolio Value (Cash)": final_value,
                "Total Profit (Cash)": final_value - self.initial_balance,
                "Total Profit (Percentage)": (final_value / self.initial_balance - 1.0) * 100.0,
                "Win Rate": 0.0,
                "Average Profit (Percentage)": 0.0,
                "Average Loss (Percentage)": 0.0,
                "Max Drawdown (Percentage)": 0.0,
                "Sigma (Period % stdev)": 0.0,
                "Num Trades": 0,
            }

        cum = (1 + ret).cumprod()
        mdd_pct = max_drawdown_from_equity(cum) * 100.0
        final_value = float(pv_s.iloc[-1])
        total_profit_cash = final_value - self.initial_balance
        total_profit_pct = (final_value / self.initial_balance - 1.0) * 100.0
        sigma_pct = float(ret.std() * 100.0)
        tr = np.asarray(trades_pnl, dtype=float)
        num_trades = int(len(tr))
        win_rate = float((tr > 0).mean()) if num_trades else 0.0
        avg_profit_cash = float(tr[tr > 0].mean()) if (tr > 0).any() else 0.0
        avg_loss_cash = float(tr[tr < 0].mean()) if (tr < 0).any() else 0.0
        avg_profit_pct = (avg_profit_cash / self.initial_balance) * 100.0 if avg_profit_cash else 0.0
        avg_loss_pct = (avg_loss_cash / self.initial_balance) * 100.0 if avg_loss_cash else 0.0

        return {
            "Final Portfolio Value (Cash)": final_value,
            "Total Profit (Cash)": total_profit_cash,
            "Total Profit (Percentage)": total_profit_pct,
            "Win Rate": win_rate,
            "Average Profit (Percentage)": avg_profit_pct,
            "Average Loss (Percentage)": avg_loss_pct,
            "Max Drawdown (Percentage)": mdd_pct,
            "Sigma (Period % stdev)": sigma_pct,
            "Num Trades": num_trades,
        }

    def plot_results(self, asset: str, style: str = "markers"):
        n = min(len(self.actual_prices), len(self.predictions), len(self.signals))
        if n == 0:
            print("Nothing to plot.")
            return
        ap = np.asarray(self.actual_prices[:n])
        pp = np.asarray(self.predictions[:n])
        ss = np.asarray(self.signals[:n], dtype=int)
        eq = np.asarray(self.portfolio_value[:n]) if hasattr(self, "portfolio_value") and len(self.portfolio_value) else None
        x = list(range(n)) if self.t_oos_index is None or len(self.t_oos_index) < n else self.t_oos_index[:n]

        fig = make_subplots(
            rows=2,
            cols=1,
            shared_xaxes=True,
            vertical_spacing=0.08,
            row_heights=[0.7, 0.3],
            subplot_titles=(f"{asset} — Prediction & Signals (OOS)", "Equity Curve"),
        )

        if style == "segments":
            for s in (-1, 0, 1):
                y = np.where(ss == s, ap, None)
                fig.add_trace(
                    go.Scatter(
                        x=x,
                        y=y,
                        mode="lines",
                        name=f"Price ({'Short' if s == -1 else 'Neutral' if s == 0 else 'Long'})",
                        line=dict(width=2, color=self.sig_colors[s]),
                        hovertemplate="%{x|%Y-%m-%d %H:%M}<br>Price %{y:.4f}<extra></extra>",
                    ),
                    row=1,
                    col=1,
                )
            fig.add_trace(
                go.Scatter(
                    x=x,
                    y=pp,
                    mode="lines",
                    name="Predicted (t+1)",
                    line=dict(width=1.2, dash="dot", color=self.line_colors["pred"]),
                    hovertemplate="%{x|%Y-%m-%d %H:%M}<br>Pred %{y:.4f}<extra></extra>",
                ),
                row=1,
                col=1,
            )
        else:
            fig.add_trace(
                go.Scatter(
                    x=x,
                    y=ap,
                    mode="lines",
                    name="Actual (t+1)",
                    line=dict(width=1.6, color=self.line_colors["actual"]),
                    hovertemplate="%{x|%Y-%m-%d %H:%M}<br>Price %{y:.4f}<extra></extra>",
                ),
                row=1,
                col=1,
            )
            fig.add_trace(
                go.Scatter(
                    x=x,
                    y=pp,
                    mode="lines",
                    name="Predicted (t+1)",
                    line=dict(width=1.2, color=self.line_colors["pred"]),
                    hovertemplate="%{x|%Y-%m-%d %H:%M}<br>Pred %{y:.4f}<extra></extra>",
                ),
                row=1,
                col=1,
            )
            for s in (-1, 0, 1):
                idx = np.where(ss == s)[0]
                if idx.size:
                    fig.add_trace(
                        go.Scatter(
                            x=[x[i] for i in idx],
                            y=ap[idx],
                            mode="markers",
                            marker=dict(size=9 if s != 0 else 7, symbol=self.sig_markers[s], color=self.sig_colors[s]),
                            name={-1: "Short/Exit", 0: "Hold Prev", 1: "Long/Enter"}[s],
                            hovertemplate="%{x|%Y-%m-%d %H:%M}<br>Price %{y:.4f}<extra></extra>",
                        ),
                        row=1,
                        col=1,
                    )

        if eq is not None and len(eq):
            fig.add_trace(
                go.Scatter(
                    x=x,
                    y=eq,
                    mode="lines",
                    name="Equity",
                    line=dict(width=1.8, color=self.line_colors["equity"]),
                    hovertemplate="%{x|%Y-%m-%d %H:%M}<br>Equity %{y:,.2f}<extra></extra>",
                ),
                row=2,
                col=1,
            )

        fig.update_layout(
            template="plotly_white",
            legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="left", x=0),
            margin=dict(l=50, r=50, t=60, b=60),
            hovermode="x unified",
        )
        fig.update_xaxes(showspikes=True, spikemode="across", spikesnap="cursor")
        fig.update_yaxes(showgrid=True, gridcolor="rgba(0,0,0,0.08)")
        fig.update_xaxes(rangeslider=dict(visible=False), row=2, col=1)
        fig.show()

    def plot_portfolio_values(self):
        if not self.portfolio_values:
            print("No portfolio values.")
            return
        
        non_empty = [pv for pv in self.portfolio_values.values() if isinstance(pv, list) and len(pv) > 0]
        if not non_empty:
            print("No portfolio values to plot.")
            return
        max_len = max(len(p) for p in non_empty)

        fig = go.Figure()
        mat = []
        for ticker, pv in self.portfolio_values.items():
            arr = pv if isinstance(pv, list) else []
            arr = list(arr)
            if len(arr) < max_len:
                arr = arr + [None] * (max_len - len(arr))
            mat.append(arr)
            fig.add_trace(
                go.Scatter(y=arr, mode="lines", name=ticker,)
            )

        mat_np = np.array([[np.nan if v is None else v for v in row] for row in mat], dtype=float)
        avg = np.nanmean(mat_np, axis=0)
        fig.add_trace(
            go.Scatter(
                y=avg,
                mode="lines",
                name="Expected Portfolio",
                line=dict(width=2, dash="dot"),
            )
        )

        fig.update_layout(
            title="Portfolio Value by Ticker",
            template="plotly_white",
            hovermode="x unified",
            legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="left", x=0),
            margin=dict(l=50, r=50, t=60, b=60),
        )
        fig.update_xaxes(rangeslider=dict(visible=False))
        fig.update_yaxes(showgrid=True, gridcolor="rgba(0,0,0,0.08)")
        fig.show()

    def show_metrics_table_plotly(self, all_performance: Dict[str, Dict[str, float]]):
        if not all_performance:
            print("No performance data.")
            return

        cols = [
            "Asset",
            "Final Portfolio Value (Cash)",
            "Total Profit (Cash)",
            "Total Profit (Percentage)",
            "Win Rate",
            "Average Profit (Percentage)",
            "Average Loss (Percentage)",
            "Max Drawdown (Percentage)",
            "Sigma (Period % stdev)",
            "Num Trades",
        ]
        rows = []
        for asset, m in all_performance.items():
            if not isinstance(m, dict):
                continue
            final_val = float(m.get("Final Portfolio Value (Cash)", 0.0))
            tot_cash = float(m.get("Total Profit (Cash)", 0.0))
            tot_pct = float(m.get("Total Profit (Percentage)", 0.0))
            win_rate = float(m.get("Win Rate", 0.0)) * 100.0  # fraction -> %
            avg_prof = float(m.get("Average Profit (Percentage)", 0.0))
            avg_loss = float(m.get("Average Loss (Percentage)", 0.0))
            mdd_pct = float(m.get("Max Drawdown (Percentage)", 0.0))
            sigma_pct = float(m.get("Sigma (Period % stdev)", 0.0))
            num_trades = int(m.get("Num Trades", 0))
            rows.append(
                [
                    asset,
                    f"{final_val:,.2f}",
                    f"{tot_cash:,.2f}",
                    f"{tot_pct:.2f}%",
                    f"{win_rate:.2f}%",
                    f"{avg_prof:.2f}%",
                    f"{avg_loss:.2f}%",
                    f"{mdd_pct:.2f}%",
                    f"{sigma_pct:.2f}%",
                    f"{num_trades}",
                ]
            )

        fig = go.Figure(
            data=[
                go.Table(
                    header=dict(values=cols, fill_color="#F5F5F5", align="left"),
                    cells=dict(values=list(zip(*rows)), align="left"),
                )
            ]
        )
        fig.update_layout(title="Performance Summary", template="plotly_white", margin=dict(l=50, r=50, t=60, b=60))
        fig.show()

    def scatter_return_sigma_with_tangent(
        self,
        all_performance: Dict[str, Dict[str, float]],
        x_key: str = "Sigma (Period % stdev)",
        y_key: str = "Total Profit (Percentage)",
        degree: int = 2,
        tangent_at: str | float = "best",
        *,
        eq_tan: Optional[pd.Series] = None,
        eq_dyn: Optional[pd.Series] = None,
        eq_labels: tuple[str, str] = ("Tangent", "MDP/HJB"),
    ):
        """Scatter Return (Y) vs Sigma (X) + poly fit + tangent line + optional portfolio points."""
        if not all_performance:
            print("No performance data.")
            return

        def _point_from_equity(eq: pd.Series) -> Optional[tuple[float, float]]:
            if eq is None or len(eq) < 3:
                return None
            eq = eq.dropna()
            if eq.empty:
                return None
            ret = eq.pct_change().dropna()
            if ret.empty:
                return None
            sigma_pct = float(ret.std() * 100.0)
            total_pct = (float(eq.iloc[-1]) / float(eq.iloc[0]) - 1.0) * 100.0
            return sigma_pct, total_pct


        xs, ys, names = [], [], []
        for asset, m in all_performance.items():
            if not isinstance(m, dict):
                continue
            x = float(m.get(x_key, np.nan))
            y = float(m.get(y_key, np.nan))
            if np.isfinite(x) and np.isfinite(y):
                xs.append(x); ys.append(y); names.append(asset)


        if len(xs) < 2:
            fig = go.Figure()
            fig.add_trace(go.Scatter(x=xs, y=ys, mode="markers",
                                    marker=dict(size=10, color=ys, colorscale="Viridis"),
                                    text=names, name="Assets",
                                    hovertemplate="%{text}<br>σ=%{x:.2f}%%<br>Return %{y:.2f}%%<extra></extra>"))
            fig.update_layout(title=f"Return vs {x_key} (not enough points for tangent)",
                            xaxis_title=x_key, yaxis_title=y_key,
                            template="plotly_white", margin=dict(l=50, r=50, t=60, b=60))
            fig.show()
            return

        degree = min(degree, len(xs) - 1)
        coeffs = np.polyfit(xs, ys, deg=degree)
        xgrid = np.linspace(min(xs), max(xs), 200)
        yfit = np.polyval(coeffs, xgrid)

        if tangent_at == "best":
            i0 = int(np.argmax(ys))
            x0, y0 = xs[i0], ys[i0]
        else:
            try:
                x0 = float(tangent_at)
            except Exception:
                x0 = float(np.median(xs))
            x0 = max(min(x0, max(xs)), min(xs))
            y0 = float(np.polyval(coeffs, x0))

        dcoeffs = np.polyder(coeffs)
        slope = float(np.polyval(dcoeffs, x0))
        ytan = y0 + slope * (xgrid - x0)

        order = np.argsort(xgrid)
        x_sorted = xgrid[order]
        yfit_sorted = yfit[order]
        ytan_sorted = ytan[order]

        fig = go.Figure()
        fig.add_trace(go.Scatter(
            x=xs, y=ys, mode="markers",
            marker=dict(size=10, color=ys, colorscale="Viridis"),
            text=[f"{n}<br>σ={x:.2f}%<br>Ret={y:.2f}%" for n, x, y in zip(names, xs, ys)],
            name="Assets", hovertemplate="%{text}<extra></extra>"
        ))
        fig.add_trace(go.Scatter(x=x_sorted, y=yfit_sorted, mode="lines", name=f"Poly fit (deg={degree})"))
        fig.add_trace(go.Scatter(x=x_sorted, y=ytan_sorted, mode="lines",
                                name=f"Tangent at σ={x0:.2f}% (slope={slope:.2f})",
                                line=dict(dash="dash")))
        fig.add_trace(go.Scatter(x=[x0], y=[y0], mode="markers",
                                name="Tangent point",
                                marker=dict(size=12, symbol="x", color="#8E24AA")))

        # ---- overlay the two portfolio points (from equity curves)
        pt_tan = _point_from_equity(eq_tan) if eq_tan is not None else None
        pt_dyn = _point_from_equity(eq_dyn) if eq_dyn is not None else None

        if pt_tan:
            fig.add_trace(go.Scatter(
                x=[pt_tan[0]], y=[pt_tan[1]], mode="markers+text",
                name=eq_labels[0], text=[eq_labels[0]], textposition="top center",
                marker=dict(size=16, symbol="star", line=dict(width=1))
            ))
        if pt_dyn:
            fig.add_trace(go.Scatter(
                x=[pt_dyn[0]], y=[pt_dyn[1]], mode="markers+text",
                name=eq_labels[1], text=[eq_labels[1]], textposition="bottom center",
                marker=dict(size=14, symbol="diamond", line=dict(width=1))
            ))

        x_all = xs + ([pt_tan[0]] if pt_tan else []) + ([pt_dyn[0]] if pt_dyn else [])
        x_min = min(x_all) if x_all else min(xs)
        x_max = max(x_all) if x_all else max(xs)
        fig.update_xaxes(range=[max(0, x_min * 0.9), x_max * 1.1])

        fig.update_layout(
            title="Return vs Sigma (Period % stdev) with Tangent Line",
            xaxis_title="Sigma (Period % stdev)",
            yaxis_title="Total Profit (Percentage)",
            template="plotly_white",
            legend=dict(orientation="h", y=1.06, x=0),
            margin=dict(l=50, r=50, t=60, b=60),
        )
        fig.show()


    def tune_hyperparameters(
        self,
        asset: str,
        n_trials: int = 30,
        timeout: Optional[int] = None,
        val_frac: float = 0.3,
        risk_aversion: float = 3,
        seed: int = 1234,
    ):
        
        #Objectivefunction = TotalProfit% - risk_aversion * |MaxDD%|
 
        df = self.data.get(asset)
        if df is None or df.empty:
            raise ValueError(f"No data for {asset}. Run fetch_data() first.")

        px = df["close"].astype(float)
        feat_rsi = rsi(px, 14)
        feats = pd.concat([px.rename("px"), feat_rsi.rename("rsi")], axis=1).dropna()

        outer_split = int(len(feats) * self.train_frac)
        if outer_split <= self.sequence_length + 2:
            raise ValueError("Not enough data for outer split.")

        base_scaler = MinMaxScaler().fit(feats.iloc[:outer_split, :])

        def build_sequences(feats_scaled: pd.DataFrame, L: int):
            X, y, y_times = [], [], []
            for i in range(L, len(feats_scaled) - 1):
                X.append(feats_scaled.iloc[i - L : i, :].values)
                y.append(feats_scaled.iloc[i + 1, 0])
                y_times.append(feats_scaled.index[i + 1])
            return (
                np.asarray(X, dtype=np.float32),
                np.asarray(y, dtype=np.float32),
                pd.Index(y_times),
            )

        tc = self.tx_cost_bps / 1e4

        def eval_trading(pred_px, true_px, px_t, long_thr, short_thr):
            exp_ret = (pred_px - px_t) / np.maximum(px_t, 1e-12)
            sig = np.zeros_like(exp_ret, dtype=int)
            sig[exp_ret > (long_thr + tc)] = 1
            sig[exp_ret < -(short_thr + tc)] = -1
            for i in range(1, len(sig)):
                if sig[i] == 0:
                    sig[i] = sig[i - 1]

            cash, qty = float(self.initial_balance), 0.0
            pv = []
            for i in range(len(sig)):
                px_fill = float(true_px[i])
                if sig[i] == 1 and qty == 0.0:
                    qty = cash / (px_fill * (1 + tc))
                    cash -= qty * px_fill * (1 + tc)
                elif sig[i] <= 0 and qty > 0.0:
                    cash += qty * px_fill * (1 - tc)
                    qty = 0.0
                pv.append(cash + qty * px_fill)

            pv = pd.Series(pv, dtype=float)
            ret = pv.pct_change().dropna()
            if ret.empty:
                return -1e9, {"Total Profit %": 0.0, "MaxDD %": 0.0, "WinRate": 0.0}
            total_pct = (pv.iloc[-1] / self.initial_balance - 1.0) * 100.0
            cum = (1 + ret).cumprod()
            mdd_pct = max_drawdown_from_equity(cum) * 100.0
            winrate = float((ret > 0).mean())
            obj = total_pct - risk_aversion * abs(mdd_pct)
            return obj, {"Total Profit %": total_pct, "MaxDD %": mdd_pct, "WinRate": winrate}

        def objective(trial: optuna.trial.Trial):
            L_low, L_high = self._sequence_bounds(len(feats), cap=512)
            L = trial.suggest_int("sequence_length", L_low, L_high, step=4)
            hidden = trial.suggest_int("hidden_layer_size", 32, 2000, step=8)
            layers = trial.suggest_int("lstm_layers", 1, 3)
            dropout = trial.suggest_float("dropout", 0.0, 0.6)
            lr = trial.suggest_float("learning_rate", 1e-5, 5e-2, log=True)
            epochs = trial.suggest_int("epochs", 10, 5000, step=10)
            long_thr = trial.suggest_float("long_threshold", 0.0, 0.003)
            short_thr = trial.suggest_float("short_threshold", 0.0, 0.003)

            feats_sc = pd.DataFrame(base_scaler.transform(feats), index=feats.index, columns=feats.columns)
            X_all, y_all, _ = build_sequences(feats_sc, L)
            n_train_outer = max(1, outer_split - L - 1)
            if n_train_outer < 30:
                raise optuna.TrialPruned()

            n_valid = max(1, int(n_train_outer * val_frac))
            n_tr = n_train_outer - n_valid

            X_tr = torch.tensor(X_all[:n_tr]).to(self.device)
            y_tr = torch.tensor(y_all[:n_tr]).to(self.device)
            X_val = torch.tensor(X_all[n_tr:n_train_outer]).to(self.device)
            y_val = torch.tensor(y_all[n_tr:n_train_outer]).to(self.device)

            px_arr = feats["px"].values
            px_t_all = px_arr[L:-1]
            px_t_val = px_t_all[n_tr:n_train_outer]

            torch.manual_seed(seed)
            model = LSTMModel(input_size=2, hidden_layer_size=hidden, output_size=1, layers=layers, dropout=dropout).to(
                self.device
            )
            optimizer = torch.optim.Adam(model.parameters(), lr=lr)
            loss_fn = nn.MSELoss()
            best_val = float("inf")
            best_state = None
            patience, patience_left = 25, 25

            for ep in range(epochs):
                model.train()
                optimizer.zero_grad()
                pred = model(X_tr).squeeze(-1)
                loss = loss_fn(pred, y_tr)
                loss.backward()
                optimizer.step()

                model.eval()
                with torch.no_grad():
                    val_pred = model(X_val).squeeze(-1)
                    val_loss = loss_fn(val_pred, y_val).item()
                    if val_loss + 1e-9 < best_val:
                        best_val = val_loss
                        best_state = {k: v.detach().clone() for k, v in model.state_dict().items()}
                        patience_left = patience
                    else:
                        patience_left -= 1
                        if patience_left <= 0:
                            break

            if best_state is not None:
                model.load_state_dict(best_state)

            with torch.no_grad():
                pred_sc = model(X_val).detach().cpu().numpy().reshape(-1, 1)
                zeros = np.zeros_like(pred_sc)
                pred_px = base_scaler.inverse_transform(np.hstack([pred_sc, zeros]))[:, 0]
                true_sc = y_val.detach().cpu().numpy().reshape(-1, 1)
                true_px = base_scaler.inverse_transform(np.hstack([true_sc, zeros]))[:, 0]

            obj, metrics = eval_trading(pred_px, true_px, px_t_val, long_thr, short_thr)
            trial.set_user_attr("metrics", metrics)
            return obj

        sampler = TPESampler(seed=seed)
        pruner = MedianPruner(n_startup_trials=10, n_warmup_steps=0)
        study = optuna.create_study(direction="maximize", sampler=sampler, pruner=pruner)
        study.optimize(objective, n_trials=n_trials, timeout=timeout, show_progress_bar=False)

        print("\n=== Best Trial ===")
        print("Value (objective):", study.best_value)
        for k, v in study.best_params.items():
            print(f"{k}: {v}")
        print("Metrics (val):", study.best_trial.user_attrs.get("metrics"))

        bp = dict(study.best_params)
        self.best_params_by_asset[asset] = bp
        self.best_metrics_by_asset[asset] = study.best_trial.user_attrs.get("metrics", {})

        self.sequence_length = bp["sequence_length"]
        self.hidden_layer_size = bp["hidden_layer_size"]
        self.lstm_layers = bp["lstm_layers"]
        self.dropout = bp["dropout"]
        self.learning_rate = bp["learning_rate"]
        self.epochs = bp["epochs"]
        self.long_threshold = bp["long_threshold"]
        self.short_threshold = bp["short_threshold"]

        self.model = LSTMModel(
            input_size=2,
            hidden_layer_size=self.hidden_layer_size,
            output_size=1,
            layers=self.lstm_layers,
            dropout=self.dropout,
        ).to(self.device)
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=self.learning_rate)
        return study

    def _sequence_bounds(self, feats_len: int, min_train_samples: int = 80, cap: int = 512) -> Tuple[int, int]:
        outer_split = int(feats_len * self.train_frac)
        max_L = outer_split - max(min_train_samples, 60)
        max_L = max(21, min(max_L, cap))
        return 21, max_L

    def apply_params(self, params: dict):
        self.sequence_length = params["sequence_length"]
        self.hidden_layer_size = params["hidden_layer_size"]
        self.lstm_layers = params["lstm_layers"]
        self.dropout = params["dropout"]
        self.learning_rate = params["learning_rate"]
        self.epochs = params["epochs"]
        self.long_threshold = params["long_threshold"]
        self.short_threshold = params["short_threshold"]


        self.model = LSTMModel(
            input_size=2,
            hidden_layer_size=self.hidden_layer_size,
            output_size=1,
            layers=self.lstm_layers,
            dropout=self.dropout,
        ).to(self.device)
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=self.learning_rate)

    def run(self, tune_each_asset: bool = False, n_trials_each: int = 30, val_frac: float = 0.2, risk_aversion: float = 4):
        if not self.data:
            self.fetch_data()

        all_perf: Dict[str, Dict[str, float]] = {}

        for asset in self.tickers:
            if asset not in self.data or self.data[asset].empty:
                print(f"Skip {asset}: no data.")
                continue

            print(f"\nProcessing {asset} ...")
    
            self.model.apply(self._reset_weights)

            if tune_each_asset:
                try:
                    if asset not in self.best_params_by_asset:
                        study = self.tune_hyperparameters(asset=asset, n_trials=n_trials_each, val_frac=val_frac, risk_aversion=risk_aversion)
                        self.best_params_by_asset[asset] = dict(study.best_params)
                        self.best_metrics_by_asset[asset] = study.best_trial.user_attrs.get("metrics", {})
                    self.apply_params(self.best_params_by_asset[asset])
                    print("[params for", asset, "]", self.best_params_by_asset[asset])
                except Exception as e:
                    print(f"[WARN] tuning {asset} failed: {e}")

            self.prepare_data(asset)
            self.train_model()
            self.make_predictions()
            self.generate_signals()
            pv = self.simulate_trading(allow_short=True)
            self.portfolio_values[asset] = pv
            perf = self.calculate_performance(allow_short=True, exec_next_bar=True)
            all_perf[asset] = perf
            self.plot_results(asset, style="segments")
        self.plot_portfolio_values()
        self.show_metrics_table_plotly(all_perf)
        self.scatter_return_sigma_with_tangent(
            all_performance=all_perf,
            x_key="Sigma (Period % stdev)",
            y_key="Total Profit (Percentage)",
            degree=2,
            tangent_at="best",
        )

    @staticmethod
    def _reset_weights(m):
        if isinstance(m, (nn.Linear, nn.LSTM)):
            m.reset_parameters()
@dataclass
class PortfolioAnalytics:
    rf_per_period: float = 0.0
    ann_factor: float = 252.0
    ridge: float = 1e-8

    @staticmethod
    def stack_close_from_tvdata(data: Dict[str, pd.DataFrame], col: str = "close") -> pd.DataFrame:
        """Stack 'close' (or col) from {symbol: DataFrame} into one price frame."""
        frames: List[pd.Series] = []
        for k, df in data.items():
            if df is None or df.empty or col not in df.columns:
                continue
            s = df[col].astype(float).rename(k)
            frames.append(s)
        if not frames:
            raise ValueError("No non-empty frames found in data.")
        px = pd.concat(frames, axis=1).sort_index()
        return px

    @staticmethod
    def returns(prices: pd.DataFrame, kind: str = "simple") -> pd.DataFrame:
        """Compute returns (simple or log)."""
        if kind == "log":
            R = np.log(prices).diff()
        elif kind == "simple":
            R = prices.pct_change()
        else:
            raise ValueError("returns(kind): choose 'simple' or 'log'")
        return R.dropna(how="any")

    def build_prices_and_returns(
        self,
        data_dict: Dict[str, pd.DataFrame],
        col: str = "close",
        kind: str = "simple",
    ) -> Tuple[pd.DataFrame, pd.DataFrame]:
        """Return (prices, returns)."""
        px = self.stack_close_from_tvdata(data_dict, col=col)
        R = self.returns(px, kind=kind)
        return px, R

    def stats(self, R: pd.DataFrame) -> Dict[str, object]:
        """Per-period & annualized stats."""
        R = R.dropna(how="any")
        mu = R.mean().values                 
        cov = R.cov().values       
        ann = self.ann_factor
        asset_mu_ann = R.mean() * ann
        asset_sig_ann = R.std() * np.sqrt(ann)
        cov_ann = cov * ann              
        return {
            "mu": mu,
            "cov": cov,
            "mu_ann": asset_mu_ann,
            "sigma_ann": asset_sig_ann,
            "cov_ann": cov_ann,
            "assets": list(R.columns),
        }

    #Tangent Part
    @staticmethod
    def _tangent_weights_closed_form(
        mu: np.ndarray,
        cov: np.ndarray,
        rf_per_period: float = 0.0,
        long_only: bool = False,
    ) -> np.ndarray:
        """Unconstrained (or long-only projected) tangent weights."""
        Sigma = cov.copy()
        mu_ex = (mu - rf_per_period).reshape(-1, )
        inv = np.linalg.pinv(Sigma)
        w = inv @ mu_ex
        if long_only:
            w = np.clip(w, 0.0, None)
        s = w.sum()
        return (w / s) if abs(s) > 0 else np.ones_like(w) / w.size

    def equity_from_static_weights(
        self,
        returns_df: pd.DataFrame,
        w: np.ndarray,
        initial_value: float = 1.0,
    ) -> pd.Series:
        """Equity given constant weights w (rebalanced each period)."""
        R = returns_df.dropna()
        w = np.asarray(w, float).reshape(-1, 1)
        if R.shape[1] != w.size:
            raise ValueError("Weight dimension mismatch.")
        port_ret = (R @ w).iloc[:, 0]
        eq = initial_value * (1.0 + port_ret).cumprod()
        return eq

    # -------------------- Deterministic frontier / tangent ------------------
    @staticmethod
    def _bounds_from_sign_policy(
        names: List[str],
        sign_policy: Optional[Dict[str, str]],
        per_asset_upper: float = 1.0,
    ) -> Tuple[np.ndarray, np.ndarray]:
        """
        sign_policy[symbol] in {'long', 'short', 'free'} -> bounds for w_i.
        Default: 'free' in [-U, +U].
        """
        n = len(names)
        lb = np.full(n, -per_asset_upper, dtype=float)
        ub = np.full(n,  per_asset_upper, dtype=float)
        if sign_policy is None:
            return lb, ub
        for i, k in enumerate(names):
            tag = sign_policy.get(k, "free")
            if tag == "long":
                lb[i] = 0.0
            elif tag == "short":
                ub[i] = 0.0
        return lb, ub

    #find feasible for understand the optimum
    def _feasible_return_range_cvxpy(self,mu: np.ndarray,lb: np.ndarray,ub: np.ndarray,l1_cap: Optional[float],) -> Tuple[float, float]:
            
            n = mu.size
            w = cp.Variable(n)
            cons = [cp.sum(w) == 1, w >= lb, w <= ub]
            if l1_cap is not None:
                cons.append(cp.norm1(w) <= l1_cap)

            prob_max = cp.Problem(cp.Maximize(mu @ w), cons)
            prob_min = cp.Problem(cp.Minimize(mu @ w), cons)

            solved = False
            for solver in ("OSQP", "ECOS", "SCS"):
                try:
                    prob_max.solve(solver=getattr(cp, solver), verbose=False)
                    prob_min.solve(solver=getattr(cp, solver), verbose=False)
                    if (w.value is not None and
                        prob_max.status in ("optimal", "optimal_inaccurate") and
                        prob_min.status in ("optimal", "optimal_inaccurate")):
                        solved = True
                        break
                except Exception:
                    continue

            if not solved:
                raise ValueError("Return range infeasible or no working solver available.")

            return float(prob_min.value), float(prob_max.value)

    def _frontier_cvxpy(self,mu: np.ndarray,cov: np.ndarray,r_targets: np.ndarray,lb: np.ndarray,ub: np.ndarray,l1_cap: Optional[float],) -> List[Tuple[float, float, np.ndarray]]:
        
        n = mu.size
        Sigma = cov + self.ridge * np.eye(n)
        out: List[Tuple[float, float, np.ndarray]] = []

        for r in r_targets:
            w = cp.Variable(n)
            cons = [cp.sum(w) == 1, (mu @ w) == r, w >= lb, w <= ub]
            if l1_cap is not None:
                cons.append(cp.norm1(w) <= l1_cap)
            obj = cp.Minimize(cp.quad_form(w, Sigma))
            prob = cp.Problem(obj, cons)

            solved = False
            for solver in ("OSQP", "ECOS", "SCS"):
                try:
                    prob.solve(solver=getattr(cp, solver), eps_abs=1e-8, eps_rel=1e-8, verbose=False)
                    if w.value is not None and prob.status in ("optimal", "optimal_inaccurate"):
                        solved = True
                        break
                except Exception:
                    continue

            if not solved:
               
                continue

            wv = np.asarray(w.value, float).ravel()
            var = float(wv @ Sigma @ wv)
            mu_p = float(mu @ wv)
            out.append((np.sqrt(var), mu_p, wv))

        if not out:
            raise ValueError("No feasible frontier points found with available solvers.")
        out = sorted(out, key=lambda t: t[0])
        return out

    def compute_frontier_and_tangent(self,R: pd.DataFrame,sign_policy: Optional[Dict[str, str]] = None,leverage_cap: Optional[float] = None,per_asset_upper: float = 1.0,n_frontier: int = 120,) -> Dict[str, object]:
        
        R = R.dropna(how="any")
        names = list(R.columns)
        mu_vec = R.mean().values   
        cov = R.cov().values          
        lb, ub = self._bounds_from_sign_policy(names, sign_policy, per_asset_upper)

        rmin, rmax = self._feasible_return_range_cvxpy(mu_vec, lb, ub, leverage_cap)
        r_targets = np.linspace(rmin, rmax, n_frontier)
        frontier = self._frontier_cvxpy(mu_vec, cov, r_targets, lb, ub, leverage_cap)

        ann = self.ann_factor
        rf_ann = self.rf_per_period * ann
        sig_f = np.array([s for s, m, w in frontier]) * np.sqrt(ann)
        mu_f =  np.array([m for s, m, w in frontier]) * ann
        W_f  =  [w for s, m, w in frontier]

        
        sharpe = (mu_f - rf_ann) / (sig_f + 1e-12)
        i_tan = int(np.argmax(sharpe))
        tan = {
            "sigma": float(sig_f[i_tan]),
            "mu":    float(mu_f[i_tan]),
            "w":     W_f[i_tan],
            "sharpe": float(sharpe[i_tan]),
        }
        return {"frontier_sigma": sig_f, "frontier_mu": mu_f, "frontier_weights": W_f, "tangent": tan, "assets": names}

    def tangent_weights(
        self,
        mu: np.ndarray,
        cov: np.ndarray,
        rf_per_period: Optional[float] = None,
        *,
        sign_policy: Optional[Dict[str, str]] = None,
        leverage_cap: Optional[float] = None,
        per_asset_upper: float = 1.0,
        long_only_fallback: bool = False,
        R_for_constraints: Optional[pd.DataFrame] = None,
        n_frontier: int = 150,
    ) -> np.ndarray:
        """
        Tangent weights. If constraints provided, compute via bounded frontier;
        else use closed-form (optionally projected long-only).
        """
        rf_pp = self.rf_per_period if rf_per_period is None else rf_per_period
        if sign_policy is None and leverage_cap is None and R_for_constraints is None:
            return self._tangent_weights_closed_form(mu, cov, rf_per_period=rf_pp, long_only=long_only_fallback)
        if R_for_constraints is None:
            raise ValueError("Provide R_for_constraints when using constrained tangent.")
        res = self.compute_frontier_and_tangent(
            R=R_for_constraints,
            sign_policy=sign_policy,
            leverage_cap=leverage_cap,
            per_asset_upper=per_asset_upper,
            n_frontier=n_frontier,
        )
        return np.asarray(res["tangent"]["w"], float)

    #Merton Part for Hamilton jacobi bellman
    def emwa(self,returns_df: pd.DataFrame,lam: float = 0.94,) -> Tuple[pd.DataFrame, List[np.ndarray], List[str]]:
        """Exponentially weighted μ_t and Σ_t."""
        R = returns_df.dropna(how="any")
        assets = list(R.columns)
        n = len(assets)
        mu_t = np.zeros((n, 1))
        M_t = np.eye(n) * 1e-8

        mu_series: List[np.ndarray] = []
        cov_series: List[np.ndarray] = []

        for _, r in R.iterrows():
            rv = r.values.reshape(-1, 1)
            mu_t = lam * mu_t + (1.0 - lam) * rv
            M_t  = lam * M_t  + (1.0 - lam) * (rv @ rv.T)
            cov_t = M_t - (mu_t @ mu_t.T)
            cov_t = cov_t + self.ridge * np.eye(n)
            mu_series.append(mu_t.flatten())
            cov_series.append(cov_t.copy())

        mu_s_df = pd.DataFrame(mu_series, index=R.index, columns=assets)
        return mu_s_df, cov_series, assets

    def hjb_dynamic(
        self,
        returns_df: pd.DataFrame,
        lam: float = 0.94,
        rf: Optional[float] = None,
        gamma_mode: str = "fixed",        # 'fixed' or 'implied_equal'
        gamma_value: float = 3.0,
        beta_gamma: float = 0.2,
        leverage_cap: Optional[float] = None,
        long_only: bool = False,
        initial_value: float = 1.0,
    ) -> Dict[str, object]:
        """Simple dynamic allocation w_t = (1/γ_t) Σ_t^{-1} (μ_t - rf)."""
        rf_pp = self.rf_per_period if rf is None else rf
        mu_s_df, cov_s, assets = self.emwa(returns_df, lam=lam)
        n = len(assets)

        # gamma_t series
        if gamma_mode == "fixed":
            gamma_ts = pd.Series([gamma_value] * len(mu_s_df), index=mu_s_df.index, name="gamma")
        elif gamma_mode == "implied_equal":
            w_eq = np.ones((n, 1)) / n
            g_s = None
            g_vals: List[float] = []
            for t in range(len(mu_s_df)):
                mu_t = mu_s_df.iloc[t].values.reshape(-1, 1)
                b_t = mu_t - rf_pp
                cov_t = cov_s[t]
                num = float(w_eq.T @ b_t)
                den = float(w_eq.T @ cov_t @ w_eq)
                g = np.nan if den <= 0 else num / max(den, 1e-12)
                g_s = g if (g_s is None or np.isnan(g_s)) else beta_gamma * g + (1 - beta_gamma) * g_s
                g_vals.append(float(g_s))
            gamma_ts = pd.Series(g_vals, index=mu_s_df.index, name="gamma")
        else:
            raise ValueError("gamma_mode must be 'fixed' or 'implied_equal'.")


        W: List[np.ndarray] = []
        for t in range(len(mu_s_df)):
            mu_t = mu_s_df.iloc[t].values.reshape(-1, 1)
            b_t = mu_t - rf_pp
            cov_t = cov_s[t]
            try:
                inv = np.linalg.inv(cov_t)
            except np.linalg.LinAlgError:
                inv = np.linalg.pinv(cov_t)
            g_t = max(float(gamma_ts.iloc[t]), 1e-12)
            w_t = (1.0 / g_t) * (inv @ b_t)

            # constraints
            w_t = w_t.flatten()
            if long_only:
                w_t = np.clip(w_t, 0.0, None)
            if leverage_cap is not None:
                lev = float(np.sum(np.abs(w_t)))
                if lev > leverage_cap and lev > 0:
                    w_t = w_t * (leverage_cap / lev)
            s = w_t.sum()
            if abs(s) > 0:
                w_t = w_t / s
            W.append(w_t)

        W_df = pd.DataFrame(W, index=mu_s_df.index, columns=assets)

        # equity from lagged weights (rebalance at t using w_{t-1})
        R = returns_df.loc[W_df.index].dropna(how="any")
        port_ret = (R * W_df.shift(1)).sum(axis=1).dropna()
        equity = initial_value * (1.0 + port_ret).cumprod()

        return {"weights": W_df, "equity": equity, "gamma": gamma_ts, "mus": mu_s_df, "covs": cov_s}
    
    def ann_mu_sigma_portfolio(eq: pd.Series, ann_factor: float)-> Tuple[float, float]:
        eq =eq.dropna()
        if len(eq) < 3:
            return 0.0, 0.0
        ret = eq.pct_change().dropna()
        if ret.empty:
            return 0.0, 0.0
        mu_ann = float(ret.mean() * ann_factor)
        sig_ann = float(ret.std() *  np.sqrt(ann_factor))
        return mu_ann, sig_ann
    

    #Plotting Part
    #draw the feasible region
    def _interior_from_frontier(self, frontier_weights: List[np.ndarray], k_between: int = 6) -> List[np.ndarray]:
        mixes: List[np.ndarray] = []
        for i in range(len(frontier_weights) - 1):
            w1, w2 = frontier_weights[i], frontier_weights[i + 1]
            for t in np.linspace(0.1, 0.9, k_between):
                mixes.append((1.0 - t) * w1 + t * w2)
        return mixes

    def plot_mpt_mdp_deterministic(
        self,
        R: pd.DataFrame,
        eq_tan: Optional[pd.Series],
        eq_dyn: pd.Series,
        sign_policy: Optional[Dict[str, str]] = None,
        leverage_cap: Optional[float] = None,
        per_asset_upper: float = 1.0,
        n_frontier: int = 120,
        interior_density: int = 6,
        rf_per_period: Optional[float] = None,
        ann_factor: Optional[float] = None,
    ) -> Dict[str, object]:
        """Plot assets, bounded efficient frontier, CML (via tangent), and MDP point."""
        rf_pp = self.rf_per_period if rf_per_period is None else rf_per_period
        ann = self.ann_factor if ann_factor is None else ann_factor
        rf_ann = rf_pp * ann

        R = R.dropna(how="any")
        names = list(R.columns)
        mu_vec = R.mean().values
        cov = R.cov().values


        res = self.compute_frontier_and_tangent(
            R=R,
            sign_policy=sign_policy,
            leverage_cap=leverage_cap,
            per_asset_upper=per_asset_upper,
            n_frontier=n_frontier,
        )
        sig_f = res["frontier_sigma"]
        mu_f = res["frontier_mu"]
        W_f = res["frontier_weights"]
        tan = res["tangent"]
        tan_sigma, tan_mu = tan["sigma"], tan["mu"]


        W_in = self._interior_from_frontier(W_f, k_between=interior_density)
        if W_in:
            Sigma = cov + self.ridge * np.eye(len(mu_vec))
            sig_in = []
            mu_in = []
            for w in W_in:
                mu_p = float(mu_vec @ w) * ann
                sig_p = float(np.sqrt(w @ Sigma @ w)) * np.sqrt(ann)
                mu_in.append(mu_p); sig_in.append(sig_p)
            sig_in = np.array(sig_in); mu_in = np.array(mu_in)
        else:
            sig_in = np.array([]); mu_in = np.array([])


        asset_mu = (R.mean() * ann)
        asset_sigma = (R.std() * np.sqrt(ann))


        dyn_ret = eq_dyn.pct_change().dropna()
        mdp_mu = float(dyn_ret.mean() * ann) if not dyn_ret.empty else 0.0
        mdp_sigma = float(dyn_ret.std() * np.sqrt(ann)) if not dyn_ret.empty else 0.0

        xmax = max([tan_sigma, mdp_sigma, (sig_in.max() if sig_in.size else 0.0), sig_f.max()]) * 1.05
        cml_x = np.linspace(0.0, xmax, 200)
        tan_sharpe = (tan_mu - rf_ann) / (tan_sigma + 1e-12)
        cml_y = rf_ann + tan_sharpe * cml_x

        # ---- Plotly
        fig = go.Figure()
        if sig_in.size:
            fig.add_trace(go.Scatter(x=sig_in, y=mu_in, mode="markers",
                                     name="Feasible Region (det.)",
                                     opacity=0.25, marker=dict(size=5)))
        fig.add_trace(go.Scatter(x=sig_f, y=mu_f, mode="lines",
                                 name="Efficient Frontier (bounded)",
                                 line=dict(width=3)))
        fig.add_trace(go.Scatter(x=cml_x, y=cml_y, mode="lines",
                                 name=f"CML (Sharpe {tan_sharpe:.2f})",
                                 line=dict(width=2, dash="dash")))
        fig.add_trace(go.Scatter(x=[tan_sigma], y=[tan_mu], mode="markers+text",
                                 name="Tangent", text=["Tangent"],
                                 textposition="top center",
                                 marker=dict(size=14, symbol="star")))
        fig.add_trace(go.Scatter(x=[mdp_sigma], y=[mdp_mu], mode="markers+text",
                                 name="MDP/HJB", text=["MDP/HJB"],
                                 textposition="bottom center",
                                 marker=dict(size=12, symbol="diamond")))
        fig.add_trace(go.Scatter(x=asset_sigma.values, y=asset_mu.values, mode="markers",
                                 name="Assets", text=names,
                                 hovertemplate="%{text}<br>σ=%{x:.2f}, μ=%{y:.2f}<extra></extra>",
                                 marker=dict(size=9)))

        fig.update_layout(
            title="MPT vs MDP — Deterministic Feasible Region & Frontier",
            xaxis_title="Volatility σ (annualized)",
            yaxis_title="Mean Return μ (annualized)",
            template="plotly_white",
            legend=dict(orientation="h", y=1.06, x=0),
            margin=dict(l=60, r=40, t=70, b=60),
        )
        fig.show()

        return {"figure": fig, "frontier_sigma": sig_f, "frontier_mu": mu_f,
                "frontier_weights": W_f, "tangent": tan}


if __name__ == "__main__":
    tickers = ["XAUUSD", "XAGUSD", "ZW1!", "AOT", "BRN1!", "S501!"]
    model = LSTMRSITradingModel(
        tickers=tickers,
        start_date="2025-01-01",
        end_date="2025-08-11",
        sequence_length=38,
        hidden_layer_size=492,
        learning_rate=0.0021,
        epochs=3000,
        lstm_layers=1,
        dropout=0.0,
        tx_cost_bps=10.0,
        long_threshold=0.0,
        short_threshold=0.0,
        train_frac=0.7,
        seed=123,
        interval=Interval.in_4_hour,
        n_bars=300,
    )

    model.fetch_data()

    an = PortfolioAnalytics(rf_per_period=0.0, ann_factor=1560.0)   # Adjust ann_factor depend of timeframe data u use like if 1 day timeframe annual is 252

    px, R = an.build_prices_and_returns(model.data, col="close", kind="simple")

   
    st = an.stats(R)
    w_tan = an.tangent_weights(st["mu"], st["cov"], long_only_fallback=True)
    eq_tan = an.equity_from_static_weights(R, w_tan, initial_value=model.initial_balance)

    
    dyn = an.hjb_dynamic(R,lam=0.94,gamma_mode="implied_equal",leverage_cap=1.0,long_only=True,initial_value=model.initial_balance,)
    W_dyn = dyn["weights"]
    eq_dyn = dyn["equity"]

    
    print("Tangent weights:", dict(zip(R.columns, w_tan)))
    print("Final equity (tangent):", float(eq_tan.iloc[-1]))
    print("Final equity (dynamic):", float(eq_dyn.iloc[-1]))

    print("[before tuning]", model.current_params())
    try:
        study = model.tune_hyperparameters(asset="XAUUSD", n_trials=1000, val_frac=0.2, risk_aversion=4)  #Optuna Tune Parameters
    except Exception as e:
        print("Tuning failed:", e)

    print("[after tuning]", model.current_params())
    model.run(tune_each_asset=True, n_trials_each=200, val_frac=0.2, risk_aversion=4) #after get params Numerical Method
    an.plot_mpt_mdp_deterministic(R=R,eq_tan=eq_tan,eq_dyn=eq_dyn,sign_policy=None,per_asset_upper=1.0,n_frontier=150)

Using GPU
Fetching data for ['XAUUSD', 'XAGUSD', 'ZW1!', 'AOT', 'BRN1!', 'S501!'] using tvDatafeed...
Fetching XAUUSD (Gold Spot) from OANDA ...
OK: XAUUSD, rows=245
Fetching XAGUSD (Silver Spot) from OANDA ...
OK: XAGUSD, rows=245
Fetching ZW1! (Wheat Futures) from CBOT ...
OK: ZW1!, rows=255
Fetching AOT (AOT) from SET ...
OK: AOT, rows=284
Fetching BRN1! (Brent Futures) from ICEEUR ...
OK: BRN1!, rows=245
Fetching S501! (SET50 Futures) from TFEX ...


C:\Users\offen\AppData\Local\Temp\ipykernel_13148\214262656.py:1049: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  R = prices.pct_change()
C:\Users\offen\AppData\Local\Temp\ipykernel_13148\214262656.py:1320: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  num = float(w_eq.T @ b_t)
C:\Users\offen\AppData\Local\Temp\ipykernel_13148\214262656.py:1321: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  den = float(w_eq.T @ cov_t @ w_eq)
[I 2025-08-23 

OK: S501!, rows=284
Tangent weights: {'XAUUSD': np.float64(0.0), 'XAGUSD': np.float64(0.4206428124851554), 'ZW1!': np.float64(0.0), 'AOT': np.float64(0.21800741617530497), 'BRN1!': np.float64(0.0), 'S501!': np.float64(0.36134977133953966)}
Final equity (tangent): 1129895.975396731
Final equity (dynamic): 1133624.587185193
[before tuning] {'sequence_length': 38, 'hidden_layer_size': 492, 'lstm_layers': 1, 'dropout': 0.0, 'learning_rate': 0.0021, 'epochs': 3000, 'tx_cost_bps': 10.0, 'long_threshold': 0.0, 'short_threshold': 0.0, 'train_frac': 0.7}


c:\Users\offen\AppData\Local\Programs\Python\Python312\Lib\site-packages\optuna\distributions.py:708: UserWarning: The distribution is specified by [21, 91] and step=4, but the range is not divisible by `step`. It will be replaced by [21, 89].
  warnings.warn(
[I 2025-08-23 16:49:08,031] Trial 0 finished with value: 0.2844489602178868 and parameters: {'sequence_length': 33, 'hidden_layer_size': 1256, 'lstm_layers': 2, 'dropout': 0.4712151502282615, 'learning_rate': 0.007675507817190953, 'epochs': 1370, 'long_threshold': 0.0008293927654292901, 'short_threshold': 0.002405616532605058}. Best is trial 0 with value: 0.2844489602178868.
c:\Users\offen\AppData\Local\Programs\Python\Python312\Lib\site-packages\optuna\distributions.py:708: UserWarning: The distribution is specified by [21, 91] and step=4, but the range is not divisible by `step`. It will be replaced by [21, 89].
  warnings.warn(
[I 2025-08-23 16:49:10,856] Trial 1 finished with value: -2.540892391084415 and parameters: {'sequen


=== Best Trial ===
Value (objective): 1.479902858207205
sequence_length: 37
hidden_layer_size: 800
lstm_layers: 3
dropout: 0.5781920902236091
learning_rate: 0.004788948670936635
epochs: 610
long_threshold: 0.0025283114114456233
short_threshold: 0.0023682367050814135
Metrics (val): {'Total Profit %': np.float64(2.659710628275902), 'MaxDD %': -0.29495194251717427, 'WinRate': 0.36}
[after tuning] {'sequence_length': 37, 'hidden_layer_size': 800, 'lstm_layers': 3, 'dropout': 0.5781920902236091, 'learning_rate': 0.004788948670936635, 'epochs': 610, 'tx_cost_bps': 10.0, 'long_threshold': 0.0025283114114456233, 'short_threshold': 0.0023682367050814135, 'train_frac': 0.7}

Processing XAUUSD ...
[params for XAUUSD ] {'sequence_length': 37, 'hidden_layer_size': 800, 'lstm_layers': 3, 'dropout': 0.5781920902236091, 'learning_rate': 0.004788948670936635, 'epochs': 610, 'long_threshold': 0.0025283114114456233, 'short_threshold': 0.0023682367050814135}
[train] seq=37 | hidden=800 | layers=3 | dropo

...


=== Best Trial ===
Value (objective): 0.0
sequence_length: 85
hidden_layer_size: 1520
lstm_layers: 3
dropout: 0.08225460715971605
learning_rate: 0.0022857341317483316
epochs: 1810
long_threshold: 0.0012081755297498626
short_threshold: 0.0006188426681734488
Metrics (val): {'Total Profit %': np.float64(0.0), 'MaxDD %': 0.0, 'WinRate': 0.0}
[params for BRN1! ] {'sequence_length': 85, 'hidden_layer_size': 1520, 'lstm_layers': 3, 'dropout': 0.08225460715971605, 'learning_rate': 0.0022857341317483316, 'epochs': 1810, 'long_threshold': 0.0012081755297498626, 'short_threshold': 0.0006188426681734488}
[train] seq=85 | hidden=1520 | layers=3 | dropout=0.08 | lr=0.002286 | epochs=1810 | thr=[long 0.0012, short 0.0006]
Epoch 150/1810  TrainLoss: 0.00359068
Epoch 300/1810  TrainLoss: 0.00354762
Epoch 450/1810  TrainLoss: 0.00361657
Epoch 600/1810  TrainLoss: 0.00359830
Epoch 750/1810  TrainLoss: 0.00367559
Epoch 900/1810  TrainLoss: 0.00365707
Epoch 1050/1810  TrainLoss: 0.00359001
Epoch 1200/1810

[I 2025-08-23 17:38:59,948] A new study created in memory with name: no-name-6745c608-724f-49a5-aad7-9e79fb31ab9c



Processing S501! ...


c:\Users\offen\AppData\Local\Programs\Python\Python312\Lib\site-packages\optuna\distributions.py:708: UserWarning:

The distribution is specified by [21, 118] and step=4, but the range is not divisible by `step`. It will be replaced by [21, 117].

[I 2025-08-23 17:39:01,456] Trial 0 finished with value: -8.154729187398047 and parameters: {'sequence_length': 37, 'hidden_layer_size': 1256, 'lstm_layers': 2, 'dropout': 0.4712151502282615, 'learning_rate': 0.007675507817190953, 'epochs': 1370, 'long_threshold': 0.0008293927654292901, 'short_threshold': 0.002405616532605058}. Best is trial 0 with value: -8.154729187398047.
c:\Users\offen\AppData\Local\Programs\Python\Python312\Lib\site-packages\optuna\distributions.py:708: UserWarning:

The distribution is specified by [21, 118] and step=4, but the range is not divisible by `step`. It will be replaced by [21, 117].

[I 2025-08-23 17:39:09,030] Trial 1 finished with value: -3.468667262318781 and parameters: {'sequence_length': 113, 'hidden_l


=== Best Trial ===
Value (objective): 0.0
sequence_length: 81
hidden_layer_size: 680
lstm_layers: 2
dropout: 0.2786585216744508
learning_rate: 0.0257515540919438
epochs: 3730
long_threshold: 0.00045890986009760357
short_threshold: 0.0015728690742659913
Metrics (val): {'Total Profit %': np.float64(0.0), 'MaxDD %': 0.0, 'WinRate': 0.0}
[params for S501! ] {'sequence_length': 81, 'hidden_layer_size': 680, 'lstm_layers': 2, 'dropout': 0.2786585216744508, 'learning_rate': 0.0257515540919438, 'epochs': 3730, 'long_threshold': 0.00045890986009760357, 'short_threshold': 0.0015728690742659913}
[train] seq=81 | hidden=680 | layers=2 | dropout=0.28 | lr=0.025752 | epochs=3730 | thr=[long 0.0005, short 0.0016]
Epoch 310/3730  TrainLoss: 0.01128104
Epoch 620/3730  TrainLoss: 0.01128104
Epoch 930/3730  TrainLoss: 0.01128104
Epoch 1240/3730  TrainLoss: 0.01128104
Epoch 1550/3730  TrainLoss: 0.01128104
Epoch 1860/3730  TrainLoss: 0.01128104
Epoch 2170/3730  TrainLoss: 0.01128104
Epoch 2480/3730  Trai

In [3]:
print("Tangent weights:", dict(zip(R.columns, w_tan)))
print("Final equity (tangent):", float(eq_tan.iloc[-1]))
print("Final equity (dynamic):", float(eq_dyn.iloc[-1]))

Tangent weights: {'XAUUSD': np.float64(0.0), 'XAGUSD': np.float64(0.4206428124851554), 'ZW1!': np.float64(0.0), 'AOT': np.float64(0.21800741617530497), 'BRN1!': np.float64(0.0), 'S501!': np.float64(0.36134977133953966)}
Final equity (tangent): 1129895.975396731
Final equity (dynamic): 1133624.587185193
